In [1]:
from sleep_analysis.datasets.d04_main_dataset import D04MainStudy
import numpy as np
import pandas as pd
import os

In [2]:
from empkins_io.sync import SyncedDataset

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

In [3]:
from biopsykit.signals.ecg import EcgProcessor

In [4]:
from pathlib import Path

In [5]:
from empkins_micro.emrad.radar import get_rpeaks, get_peak_probabilities

In [6]:
from fau_colors import cmaps, register_fausans_font

In [7]:
%matplotlib widget

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

In [10]:
cwd = Path('.')

files = sorted([file for file in cwd.glob("../**/*MAE_gridsearch*") if '.ipynb_checkpoints' not in file.parts])


In [11]:
gridsearch_results = pd.DataFrame() 
for file in files:
    res = pd.read_csv(file, index_col=0)
    gridsearch_results = pd.concat([gridsearch_results, res], axis=0)

In [12]:
gridsearch_results = gridsearch_results.T
gridsearch_results

,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate
0.05,31.529987,28.587458,38.721431,25.823367,11.434243,11.410585,29.108398,25.328576,46.627691,32.886763,19.117615,41.915933,35.561062,38.259295,15.477115,29.350540,29.444770,26.034915,26.673111,25.258213,12.913308,25.703646,31.785092,5.456481,22.575923,12.545932,12.192840,25.968335,48.071784,25.208166,23.278229,34.680763,28.211277,25.230763,37.142152,23.812371,10.838488,10.936784,25.171848,22.792070,43.091458,29.253539,18.121868,25.628369,21.600022,36.019322,20.362309,9.389817,8.122095,21.228447,19.944733,38.591256,25.947538,15.588772,35.262379,31.520512,28.579770,38.665968,25.828574,11.435486,11.413369,29.105364,25.322654,46.606877,32.886756,19.118041,23.196743,19.001352,35.331800,18.149934,8.745579,5.064442,18.931369,17.764906,36.076107,23.781890
0.075,21.814064,20.061864,27.610080,18.377067,7.548585,6.430725,19.263468,17.027292,35.467263,23.299820,11.852370,30.799446,25.025428,26.832356,10.919279,20.519138,20.730836,16.614455,16.994940,16.804351,7.259252,18.756246,22.373915,2.962246,15.012980,7.182449,8.258186,17.880631,36.935711,16.899338,15.891162,25.351893,19.798027,17.576143,27.146660,16.575147,6.882453,6.739522,16.254232,15.708991,32.055674,20.439972,12.167794,18.011246,15.104601,26.522958,14.104117,6.059241,5.909693,13.955676,14.118352,29.437234,18.072066,10.295191,25.653938,21.818156,20.069356,27.519376,18.367067,7.541518,6.426193,19.238703,17.017069,35.444429,23.299820,11.852512,16.158845,13.302418,25.450172,12.343876,5.864753,2.677691,12.589486,12.316349,27.271130,16.771134
0.1,16.934258,15.710989,21.396731,14.331639,5.689471,4.434454,14.456017,13.119434,29.394984,18.376481,9.499739,24.911574,19.462783,20.876952,8.273387,15.735409,16.008439,11.929503,11.820216,12.827437,5.181263,15.301220,17.448294,2.079806,11.011320,4.984065,6.100498,13.566509,30.892049,12.743794,12.086819,20.220181,15.445588,13.666984,21.400558,13.001835,5.382655,5.029439,12.024088,12.367998,26.955072,16.257369,9.056900,14.402993,11.908587,20.708429,10.990162,4.684612,4.382254,10.434418,11.204313,24.627876,14.103453,7.902007,20.545098,16.949904,15.711279,21.309583,14.328971,5.693200,4.455861,14.438865,13.119434,29.350986,18.376480,9.499746,13.072096,10.476918,20.331776,9.606945,4.500410,2.866538,9.639479,10.176367,23.052267,13.215866
0.125,13.375905,12.660486,17.369372,11.393476,4.430950,3.838081,11.214207,10.512118,25.024631,14.558905,7.489238,20.297298,15.752836,16.498089,6.419383,12.454932,12.769114,8.972808,8.388329,9.750420,3.983448,12.718633,13.836113,1.530126,8.371625,3.608313,4.749947,10.819781,25.886755,9.652489,9.520426,16.190691,12.245324,10.980363,17.925290,10.595105,4.348767,3.740847,9.330901,10.101803,23.294909,13.202959,6.905661,11.905482,9.189074,16.786088,8.918501,3.833771,3.276239,8.314604,8.739756,21.516218,11.554031,6.119739,16.841785,13.378804,12.660177,17.263587,11.385831,4.431810,3.844798,11.197651,10.511904,24.982323,14.558905,7.489141,10.726849,8.566022,16.685542,7.578790,3.687015,2.698898,7.429550,8.082147,20.381108,10.566067
0.15,10.688904,10.203699,15.001897,9.221267,3.553260,2.947715,8.599593,8.309419,21.522744,11.553574,6.044612,16.322052,12.449036,13.116123,4.949924,9.745283,9.999122,6.528667,6

In [13]:
gridsearch_results.agg(["mean", "std"], axis=1)

,mean,std
0.05,25.101931,10.163977
0.075,17.509024,7.907421
0.1,13.729755,6.565670
0.125,11.076488,5.595273
0.15,9.019777,4.863311
0.175,7.189909,4.134769
0.2,5.532511,3.445117
0.225,4.080590,2.798590
0.25,3.089350,2.301788
0.255,2.905409,2.057046


In [14]:
### Compare different window sizes

In [29]:
files = sorted([file for file in cwd.glob("../**/*MAE_gridsearch_subj30*")])
files_30 = files[12:]
files_300 = files[:11]
files_120 = sorted([file for file in cwd.glob("../**/*MAE_gridsearch_subj120*")])
files_500 = sorted([file for file in cwd.glob("../**/*MAE_gridsearch_subj500*")])

In [32]:
gridsearch_results = pd.DataFrame() 
for file in files_30:
    res = pd.read_csv(file, index_col=0)
    gridsearch_results = pd.concat([gridsearch_results, res], axis=0)
gridsearch_results = gridsearch_results.T
gridsearch_results

,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate
0.05,31.520512,28.579770,38.665968,25.828574,11.435486,11.413369,29.105364,25.322654,46.606877,32.886756,19.118041
0.075,21.818156,20.069356,27.519376,18.367067,7.541518,6.426193,19.238703,17.017069,35.444429,23.299820,11.852512
0.1,16.949904,15.711279,21.309583,14.328971,5.693200,4.455861,14.438865,13.119434,29.350986,18.376480,9.499746
0.125,13.378804,12.660177,17.263587,11.385831,4.431810,3.844798,11.197651,10.511904,24.982323,14.558905,7.489141
0.15,10.672722,10.200014,14.901360,9.225634,3.553268,2.948206,8.592373,8.301957,21.455123,11.553573,6.044432
0.175,8.169270,7.915105,11.964649,7.053472,2.879693,2.208977,6.259493,6.595413,17.893711,9.584046,4.636635
0.2,5.734445,5.430383,8.318303,5.373101,2.390554,1.410663,4.266475,5.146979,15.185699,7.533181,3.409289
0.225,4.642107,3.456507,5.665113,3.447220,2.157508,1.158446,2.626588,3.281067,12.337872,5.588575,2.231152
0.25,3.283900,2.465141,3.832249,2.200046,1.676719,0.974375,1.594098,2.407912,10.184368,4.950949,1.560445
0.255,3.230900,2.454715,3.396446,1.733167,1.572040,1.088385,1.450475,2.404602,8.594131,4.779404,1.519954


In [33]:
gridsearch_results.agg(["mean", "std"], axis=1)

,mean,std
0.05,27.316670,10.643997
0.075,18.963109,8.458319
0.1,14.839483,7.025217
0.125,11.973175,5.937096
0.15,9.768060,5.195735
0.175,7.741860,4.386113
0.2,5.836279,3.703107
0.225,4.235650,3.036772
0.25,3.193655,2.587697
0.255,2.929474,2.171588


In [34]:
gridsearch_results = pd.DataFrame() 
for file in files_120:
    res = pd.read_csv(file, index_col=0)
    gridsearch_results = pd.concat([gridsearch_results, res], axis=0)
gridsearch_results = gridsearch_results.T
gridsearch_results

,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate
0.05,28.211277,25.230763,37.142152,23.812371,10.838488,10.936784,25.171848,22.792070,43.091458,29.253539,18.121868
0.075,19.798027,17.576143,27.146660,16.575147,6.882453,6.739522,16.254232,15.708991,32.055674,20.439972,12.167794
0.1,15.445588,13.666984,21.400558,13.001835,5.382655,5.029439,12.024088,12.367998,26.955072,16.257369,9.056900
0.125,12.245324,10.980363,17.925290,10.595105,4.348767,3.740847,9.330901,10.101803,23.294909,13.202959,6.905661
0.15,10.074513,8.955615,14.473430,8.513981,3.402870,2.743877,7.305804,8.109449,20.618806,11.044780,5.440836
0.175,7.833474,6.837409,11.731149,6.625684,2.953284,1.911293,5.582246,6.390415,18.235047,8.810168,4.286257
0.2,5.944304,4.691832,8.998719,4.946054,2.280848,1.260862,3.828105,5.075560,15.300303,7.034242,3.293041
0.225,4.189963,3.082399,5.844511,3.603853,1.883057,1.070094,2.675366,3.499907,13.577073,5.171796,2.504290
0.25,3.036001,2.325599,3.242136,2.126321,1.490305,0.926081,1.837821,2.582510,9.739732,4.222256,1.681751
0.255,3.688797,2.205991,2.973065,2.067360,1.481470,0.949172,1.725205,2.404458,7.618097,4.125473,1.458786


In [35]:
gridsearch_results.agg(["mean", "std"], axis=1)

,mean,std
0.05,24.963874,9.773038
0.075,17.394965,7.624317
0.1,13.689862,6.446459
0.125,11.151993,5.680924
0.15,9.153087,5.070199
0.175,7.381493,4.507371
0.2,5.695806,3.841299
0.225,4.282028,3.375847
0.25,3.019137,2.409045
0.255,2.790716,1.869601


In [36]:
gridsearch_results = pd.DataFrame() 
for file in files_300:
    res = pd.read_csv(file, index_col=0)
    gridsearch_results = pd.concat([gridsearch_results, res], axis=0)
gridsearch_results = gridsearch_results.T
gridsearch_results

,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate
0.05,25.628369,21.600022,36.019322,20.362309,9.389817,8.122095,21.228447,19.944733,38.591256,25.947538,15.588772
0.075,18.011246,15.104601,26.522958,14.104117,6.059241,5.909693,13.955676,14.118352,29.437234,18.072066,10.295191
0.1,14.402993,11.908587,20.708429,10.990162,4.684612,4.382254,10.434418,11.204313,24.627876,14.103453,7.902007
0.125,11.905482,9.189074,16.786088,8.918501,3.833771,3.276239,8.314604,8.739756,21.516218,11.554031,6.119739
0.15,9.222925,7.856627,13.722734,7.186357,3.196992,2.481560,6.740728,7.188136,19.393166,9.525776,4.789258
0.175,7.482062,6.176592,11.241831,5.779532,2.761819,1.729749,5.045060,5.946944,17.390052,7.967559,3.770914
0.2,6.116812,4.654656,7.938092,4.471443,2.346704,1.207792,3.918876,5.049027,15.011840,6.355801,3.319093
0.225,4.541501,2.898720,5.482824,2.936580,1.734629,0.871935,2.814718,3.233601,11.717647,5.035285,2.261314
0.25,2.849214,2.151674,3.011857,1.824307,1.500177,0.909469,1.689620,2.251717,10.389373,4.060841,1.735086
0.255,3.496251,1.974766,2.693575,1.554664,1.294849,0.910580,1.588586,2.073564,8.786833,4.125950,1.555165


In [37]:
gridsearch_results.agg(["mean", "std"], axis=1)

,mean,std
0.05,22.038425,9.491687
0.075,15.599125,7.369916
0.1,12.304464,6.139462
0.125,10.013955,5.378306
0.15,8.300387,4.818125
0.175,6.844738,4.360191
0.2,5.490012,3.675999
0.225,3.957159,2.922009
0.25,2.943030,2.613697
0.255,2.732253,2.228408


In [38]:
gridsearch_results = pd.DataFrame() 
for file in files_500:
    res = pd.read_csv(file, index_col=0)
    gridsearch_results = pd.concat([gridsearch_results, res], axis=0)
gridsearch_results = gridsearch_results.T
gridsearch_results

,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate,Heart_Rate
0.05,23.196743,19.001352,35.331800,18.149934,8.745579,5.064442,18.931369,17.764906,36.076107,23.781890
0.075,16.158845,13.302418,25.450172,12.343876,5.864753,2.677691,12.589486,12.316349,27.271130,16.771134
0.1,13.072096,10.476918,20.331776,9.606945,4.500410,2.866538,9.639479,10.176367,23.052267,13.215866
0.125,10.726849,8.566022,16.685542,7.578790,3.687015,2.698898,7.429550,8.082147,20.381108,10.566067
0.15,8.722151,7.007133,13.570686,6.198484,3.251696,2.169421,6.072853,6.698171,18.145952,8.979230
0.175,6.234622,5.940354,10.741780,4.912121,2.703525,1.650892,4.663035,5.659601,15.949587,7.212391
0.2,4.713428,4.630076,8.285603,3.660373,2.386317,1.267295,3.686819,4.463541,14.527990,5.813259
0.225,3.515677,3.319210,5.682123,2.433546,1.803420,0.795099,2.419714,3.480062,12.079976,4.719762
0.25,2.624767,2.001762,3.005934,1.565739,1.500543,0.864994,1.605894,2.634112,8.888156,3.962186
0.255,2.623813,1.945395,2.881024,1.283241,1.544407,0.867070,1.514848,2.207051,8.683037,3.920708


In [39]:
gridsearch_results.agg(["mean", "std"], axis=1)

,mean,std
0.05,20.604412,9.858243
0.075,14.474586,7.599323
0.1,11.693866,6.234936
0.125,9.640199,5.409681
0.15,8.081578,4.732496
0.175,6.566791,4.118563
0.2,5.343470,3.739425
0.225,4.024859,3.156004
0.25,2.865409,2.297184
0.255,2.747059,2.266238
